# Prepare Annotation Tables

In [1]:
import pandas as pd
import os
import math
import Bio
from Bio import Seq
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.Data.CodonTable
table = Bio.Data.CodonTable.ambiguous_dna_by_id[1]

### Get variant Data

In [62]:
variants = pd.read_csv('../4-Variation/variant_list.05.tsv', sep='\t')
#variants_no_filter = pd.read_csv('../4-Variation/variant_list.tsv', sep='\t')

In [63]:
synonymous=variants[(variants.EFFECT!='SYNONYMOUS_CODING') & (variants.EFFECT!='.')]

In [64]:
synonymous.GENE.unique()

array(['S', 'orf1ab', 'N', 'E', 'ORF3a', 'ORF7a', 'ORF10', 'ORF6', 'ORF8',
       'M', 'ORF7b'], dtype=object)

In [65]:
variants.GENE.unique()

array(['.', 'orf1ab', 'S', 'N', 'E', 'ORF3a', 'ORF7a', 'ORF6', 'ORF10',
       'ORF8', 'M', 'ORF7b'], dtype=object)

In [66]:
synonymous.groupby('POS')['ALT'].value_counts()


POS    ALT
269    T      1
275    T      1
285    T      4
308    A      2
312    A      1
             ..
29516  A      1
29580  A      1
29612  C      1
29625  T      1
29642  T      3
Name: ALT, Length: 1161, dtype: int64

In [67]:
synonymous['NbSample']=0
synonymous['MeanAF']=0
groups=synonymous.groupby('POS')['ALT'].value_counts()
meanAF=synonymous.groupby(['POS','ALT'])['AF'].mean()
for i,row in synonymous.iterrows():
    synonymous.loc[i, 'NbSample']=groups[row['POS']][row['ALT']]
    synonymous.loc[i, 'MeanAF']=meanAF[row['POS']][row['ALT']]

<ipython-input-67-950837d7ae45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['NbSample']=0
<ipython-input-67-950837d7ae45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['MeanAF']=0
/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [68]:
variants['NbSample']=0
variants['MeanAF']=0
groups=variants.groupby('POS')['ALT'].value_counts()
meanAF=variants.groupby(['POS','ALT'])['AF'].mean()
for i,row in variants.iterrows():
    variants.loc[i, 'NbSample']=groups[row['POS']][row['ALT']]
    variants.loc[i, 'MeanAF']=meanAF[row['POS']][row['ALT']]

In [69]:
variants

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
0,SRR11578269,NC_045512.2,2668,C,T,6019,0.050008,1,"3670,2007,206,107",MODIFIER,NONE,.,.,.,S,5,0.093522
1,SRR11578028,NC_045512.2,17239,C,T,6139,0.050008,1,"3566,2191,200,127",MODIFIER,NONE,.,.,.,S,9,0.098220
2,SRR11578268,NC_045512.2,26010,C,T,6579,0.050008,7,"4929,1301,276,60",MODIFIER,NONE,.,.,.,S,14,0.087375
3,SRR11578094,NC_045512.2,12050,A,T,5159,0.050010,13,"2507,2381,155,114",MODIFIER,NONE,.,.,.,S,2,0.172133
4,SRR11578157,NC_045512.2,8739,C,T,3079,0.050016,2,"1963,625,125,35",MODIFIER,NONE,.,.,.,S,10,0.054687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68561,SRR11494648,NC_045512,28657,C,T,177,1.000000,0,"0,0,75,102",LOW,SILENT,SYNONYMOUS_CODING,N,gaC/gaT,S,52,0.737106
68562,SRR11605107,NC_045512.2,28881,G,A,102,1.000000,0,"0,0,44,58",MODIFIER,NONE,.,.,.,S,120,0.896983
68563,SRR11605107,NC_045512.2,28882,G,A,103,1.000000,0,"0,0,44,59",MODIFIER,NONE,.,.,.,S,118,0.881007
68564,SRR11605107,NC_045512.2,28883,G,C,102,1.000000,0,"0,0,43,59",MODIFIER,NONE,.,.,.,S,118,0.917067


In [70]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
371,SRR11059946,NC_045512,21641,G,C,3780,0.050265,42,"626,2947,14,182",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gca/Cca,S,1,0.050265
396,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gtt/Ttt,S,1,0.050279
397,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Ggc/Tgc,S,1,0.050279
455,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Tct/Cct,S,1,0.050314
523,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,cTc/cCc,S,1,0.050360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68544,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68545,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68546,SRR11494718,NC_045512,25563,G,T,125,1.000000,0,"0,0,52,73",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68547,SRR11494732,NC_045512,25563,G,T,135,1.000000,0,"0,0,63,72",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741


### Get Variant Selection Data

In [71]:
selection_data = pd.read_csv('data.csv', sep=',')

In [72]:
#selection_data = pd.read_csv('Selective_Analysis_Data/20200418.csv', sep=',')

In [73]:
selection_data.loc[6]

First Detected                                                [["20200403","20200408","20200410","20200412",...
Site                                                          N 13<br/>Genomic <a target = '_blank_' href = ...
Categories                                                    {"fel":true,"meme":true,"fraction":0.99999999,...
Site composition/Evolutionary Credibility                     {"composition":"<span style=\"color:darkgreen\...
Substitutions at <br> Internal Branches <br> [Syn:Non-syn]                                                [0,2]
Name: 6, dtype: object

Clean up of data, split collumns and removing html tags

In [74]:
selection_data['syn']=0
selection_data['non_syn']=0
selection_data['REF']=0
selection_data['ALT']=0
selection_data['Genomic_coord_end']=0
selection_data['Genomic_coord_start']=0
selection_data['Gene']=0
selection_data['Main_ref_codon']=0
selection_data['Main_alt_codon']=0
selection_data['Main_ref_residue']=0
selection_data['Main_alt_residue']=0
selection_data['NbSample']=0
selection_data['MeanAF']=0


In [75]:
for i,row in selection_data.iterrows():
    selection_data.loc[i,'syn'] =row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[0]
    selection_data.loc[i, 'non_syn']=row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[1]
    selection_data.loc[i, 'Genomic_coord_start']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[0]
    selection_data.loc[i, 'Genomic_coord_end']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[1]
    selection_data.loc[i, 'NbSample']=len(row['First Detected'].replace('[[','').replace("],9]",'').split(','))
    locations=selection_data['Site'][i][:-4].split('<br/>')[0].split('<br>')
    selection_data.loc[i, 'Gene']=",".join(locations)
    composition=selection_data['Site composition/Evolutionary Credibility'][i].split(',')[2:]
    if len(composition)>0:
        variation=list((np.unique(np.array([i.replace('"',"") for i in composition if '/' in i]))))
        selection_data.loc[i, 'Main_ref_codon']=list(variation)[0].split('/')[0]
        selection_data.loc[i, 'Main_alt_codon']=','.join([i.split('/')[1] for i in variation])
        selection_data.loc[i, 'Main_ref_residue']=Bio.Seq._translate_str(list(variation)[0].split('/')[0], table)
        selection_data.loc[i, 'Main_alt_residue']=','.join([Bio.Seq._translate_str(i.split('/')[1], table) for i in variation])
    else :
        variation=selection_data['Site composition/Evolutionary Credibility'][i].replace('{"composition":"','').replace("<span title = \'\'>",'').replace('</sub></span>","variants":[]}','').split('</sub></span>')
        dic={}
        variations=''
        for j in variation:
            var=re.sub("\<span.*\\'\>",'',j)
            dic[var.split('<sub>')[0]]=var.split('<sub>')[1] 
            variations=variations+var.split('<sub>')[0]+','
        selection_data.loc[i, 'Main_ref_residue']=variations[0]
        selection_data.loc[i, 'Main_alt_residue']=variations[1:].strip(',')
    
 

Remove unecessary columns

In [76]:
selection_data.head()

,First Detected,Site,Categories,Site composition/Evolutionary Credibility,Substitutions at <br> Internal Branches <br> [Syn:Non-syn],syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,Main_alt_residue,NbSample,MeanAF
0,"[[""20200325"",""20200403"",""20200404"",""20200408"",...",ORF1b 313<br>RdRp 323<br/>Genomic <a target = ...,"{""fel"":true,""meme"":true,""fraction"":0.994145046...","{""composition"":""<span style=\""color:#c2a90f\""t...","[0,2]",0,2,0,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,L,11,0
1,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF3a 99<br/>Genomic <a target = '_blank_' hre...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]",0,1,0,0,25689,25687,ORF3a 99,Ggc,Agc,G,S,12,0
2,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 265<br>nsp2 85<br/>Genomic <a target = '...,"{""fel"":true,""meme"":true,""fraction"":0.990797444...","{""composition"":""<span style=\""color:darkgreen\...","[0,2]",0,2,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,T,12,0
3,"[[""20200410"",""20200412"",""20200414"",""20200415"",...",ORF1b 87<br>RdRp 97<br/>Genomic <a target = '_...,"{""fel"":true,""meme"":true,""fraction"":0.994634214...","{""composition"":""<span style=\""color:darkgreen\...","[0,2]",0,2,0,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,V,7,0
4,"[[""20200325""],11]",ORF1b 1426<br>helicase 504<br/>Genomic <a targ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,2]",0,2,0,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,L,2,0


In [77]:
smaller=selection_data[['syn','non_syn','REF','ALT','Genomic_coord_end','Genomic_coord_start','Gene','Main_ref_codon','Main_alt_codon','Main_ref_residue','Main_alt_residue','NbSample','MeanAF']]

In [78]:
smaller.loc[6]

syn                        0
non_syn                    2
REF                        0
ALT                        0
Genomic_coord_end      28312
Genomic_coord_start    28310
Gene                    N 13
Main_ref_codon           Cgc
Main_alt_codon           Ggc
Main_ref_residue           R
Main_alt_residue           G
NbSample                   9
MeanAF                     0
Name: 6, dtype: object

## Add Feature data  

Main Feature , eg. ORF1a, and secondary Feature, eg. nsp5

In [79]:
smaller['MainFeat']='NA'
smaller['MainResidue']=0
smaller['SecondFeat']='NA'
smaller['SecondResidue']=0
smaller['ThirdFeat']='NA'
smaller['ThirdResidue']=0


smaller['MainResidue']='na'
smaller['MainFeat']='na'
smaller['SecondFeat']='na'
smaller['SecondResidue']='na'
smaller['Second_ref_codon']='na'
smaller['Second_alt_codon']='na'
smaller['Second_ref_residue']='na'
smaller['Second_alt_residue']='na'
smaller['ThirdResidue']='na'
smaller['ThirdFeat']='na'
smaller['Third_ref_codon']='na'
smaller['Third_alt_codon']='na'
smaller['Third_ref_residue']='na'
smaller['Third_alt_residue']='na'

<ipython-input-79-d922c7532572>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainFeat']='NA'
<ipython-input-79-d922c7532572>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainResidue']=0
<ipython-input-79-d922c7532572>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [80]:
for i,row in smaller.iterrows():
    liste_loc=row['Gene'].split(',')
    smaller.loc[i, 'MainFeat']=" ".join(liste_loc[0].split(" ")[:-1])
    smaller.loc[i, 'MainResidue']=liste_loc[0].split(" ")[-1]
    if len(liste_loc)>1:   
        smaller.loc[i, 'SecondFeat']=" ".join(liste_loc[1].split(" ")[:-1])
        smaller.loc[i, 'SecondResidue']=liste_loc[1].split(" ")[-1]
    if len(liste_loc)>2:
        print(liste_loc)
        smaller.loc[i, 'ThirdFeat']=" ".join(liste_loc[2].split(" ")[:-1])
        smaller.loc[i, 'ThirdResidue']=liste_loc[2].split(" ")[-1]

In [81]:
liste_loc

['ORF1b 2612', "2'-O-ribose methyltransferase 216"]

In [82]:
smaller

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,ThirdFeat,ThirdResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
0,0,2,0,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na
1,0,1,0,0,25689,25687,ORF3a 99,Ggc,Agc,G,...,na,na,na,na,na,na,na,na,na,na
2,0,2,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,...,na,na,na,na,na,na,na,na,na,na
3,0,2,0,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,...,na,na,na,na,na,na,na,na,na,na
4,0,2,0,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0,0,0,0,17799,17797,"ORF1b 1443,helicase 521",0,0,V,...,na,na,na,na,na,na,na,na,na,na
127,0,1,0,0,17886,17884,"ORF1b 1472,helicase 550",0,0,T,...,na,na,na,na,na,na,na,na,na,na
128,0,0,0,0,18984,18982,"ORF1b 1838,3'-to-5' exonuclease 315",0,0,M,...,na,na,na,na,na,na,na,na,na,na
129,0,1,0,0,21243,21241,"ORF1b 2591,2'-O-ribose methyltransferase 195",0,0,T,...,na,na,na,na,na,na,na,na,na,na


In [83]:
#workingtable=smaller.drop(labels='Gene', axis=1)

In [84]:
workingtable=smaller.copy()

In [85]:
workingtable.SecondFeat.unique()

array(['RdRp', 'na', 'nsp2', 'helicase', 'nsp3', "3'-to-5' exonuclease",
       "2'-O-ribose methyltransferase", '3C-like proteinase', 'nsp6',
       'endoRNAse', 'leader protein', 'nsp4'], dtype=object)

In [86]:
workingtable.MainFeat.unique()

array(['ORF1b', 'ORF3a', 'ORF1a', 'S', 'N', 'ORF6', 'ORF8', 'M'],
      dtype=object)

In [141]:
workingtable.head(20)

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,POS,AF,type
0,0,2,na,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,...,na,na,na,na,na,na,na,14407,0,S
1,0,1,na,0,25689,25687,ORF3a 99,Ggc,Agc,G,...,na,na,na,na,na,na,na,25687,0,S
2,0,2,na,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,...,na,na,na,na,na,na,na,1058,0,S
3,0,2,na,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,...,na,na,na,na,na,na,na,13729,0,S
4,0,2,na,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,...,na,na,na,na,na,na,na,17746,0,S
5,0,3,na,0,23404,23402,S 614,0,0,G,...,na,na,na,na,na,na,na,23402,0,S
6,0,2,na,0,28312,28310,N 13,Cgc,Ggc,R,...,na,na,na,na,na,na,na,28310,0,S
7,0,1,na,0,25563,25561,ORF3a 57,caG,"caA,caT",Q,...,na,na,na,na,na,na,na,25561,0,S
8,0,2,na,0,4051,4049,"ORF1a 1262,nsp3 444",Ctt,Ttt,L,...,na,na,na,na,na,na,na,4049,0,S
9,0,2,na,0,17193,17191,"ORF1b 1241,helicase 319",tgT,tgC,C,...,na,na,na,na,na,na,na,17191,0,S


In [142]:
workingtable.to_csv('selection_var.csv',  sep='\t',index=False, header=True)

In [87]:
dic={}

In [88]:
for i in workingtable['MainFeat'].unique():
    dic[i]=workingtable[workingtable['MainFeat']==i]

In [89]:
dic['ORF1b']

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,ThirdFeat,ThirdResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
0,0,2,0,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na
3,0,2,0,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,...,na,na,na,na,na,na,na,na,na,na
4,0,2,0,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na
9,0,2,0,0,17193,17191,"ORF1b 1241,helicase 319",tgT,tgC,C,...,na,na,na,na,na,na,na,na,na,na
10,0,2,0,0,18738,18736,"ORF1b 1756,3'-to-5' exonuclease 233",Ttt,Ctt,F,...,na,na,na,na,na,na,na,na,na,na
11,0,2,0,0,21138,21136,"ORF1b 2556,2'-O-ribose methyltransferase 160",aAg,aGg,K,...,na,na,na,na,na,na,na,na,na,na
31,0,3,0,0,13515,13513,"ORF1b 15,RdRp 25",0,0,G,...,na,na,na,na,na,na,na,na,na,na
32,0,1,0,0,13629,13627,"ORF1b 53,RdRp 63",Gac,Tac,D,...,na,na,na,na,na,na,na,na,na,na
33,0,1,0,0,15279,15277,"ORF1b 603,RdRp 613",Cac,Tac,H,...,na,na,na,na,na,na,na,na,na,na
34,0,1,0,0,17826,17824,"ORF1b 1452,helicase 530",aCt,aTt,T,...,na,na,na,na,na,na,na,na,na,na


## Import Feature data from NCBI Genbank File

In [90]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO


In [91]:
recs = [rec for rec in SeqIO.parse("annotations.gb", "genbank")]


Get Features in a Dataframe, and add ORF1b that is not in the GenBank File

In [92]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)

In [93]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [print(feat.location) for feat in rec.features if feat.type == "CDS"]

join{[265:13468](+), [13467:21555](+)}
[265:13483](+)
[21562:25384](+)
[25392:26220](+)
[26244:26472](+)
[26522:27191](+)
[27201:27387](+)
[27393:27759](+)
[27755:27887](+)
[27893:28259](+)
[28273:29533](+)
[29557:29674](+)


In [94]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555},{'gene': 'RdRp1', 'start': 13441, 'end': 13468},{'gene': 'RdRp2', 'start': 13467, 'end': 16236}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide" or feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)


In [95]:
features_mat=pd.DataFrame(features_list)

In [96]:
features_mat=features_mat.drop_duplicates().sort_values(by=['start']).reset_index()[['gene','start','end']]

In [97]:
features_mat['short']='na'
for i,row in features_mat.iterrows():
    features_mat.loc[i, 'short']=row['gene'].split(' ')[0]



In [98]:
features_mat

,gene,start,end,short
0,ORF1a polyprotein,265,13483,ORF1a
1,ORF1ab polyprotein,265,21555,ORF1ab
2,leader protein,265,805,leader
3,nsp2,805,2719,nsp2
4,nsp3,2719,8554,nsp3
5,nsp4,8554,10054,nsp4
6,3C-like proteinase,10054,10972,3C-like
7,nsp6,10972,11842,nsp6
8,nsp7,11842,12091,nsp7
9,nsp8,12091,12685,nsp8


Removing ORF1ab From the table

In [99]:
features_mat=features_mat.drop(labels=[1,13,14], axis=0)

In [100]:
features_mat['Length']=(features_mat['end']-features_mat['start'])/3
features_mat['Length']=features_mat['Length'].astype('int64')

In [101]:
features_mat

,gene,start,end,short,Length
0,ORF1a polyprotein,265,13483,ORF1a,4406
2,leader protein,265,805,leader,180
3,nsp2,805,2719,nsp2,638
4,nsp3,2719,8554,nsp3,1945
5,nsp4,8554,10054,nsp4,500
6,3C-like proteinase,10054,10972,3C-like,306
7,nsp6,10972,11842,nsp6,290
8,nsp7,11842,12091,nsp7,83
9,nsp8,12091,12685,nsp8,198
10,nsp9,12685,13024,nsp9,113


Get Genome sequence to get codon informations

In [102]:
!head sequence.fasta


>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT
GCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTC
GTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCT


In [103]:
with open('sequence.fasta') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [104]:
gen_seq=''.join(content[1:])

In [105]:
len(gen_seq)

29903

In [106]:
266+4382*3

13412

In [107]:
gen_seq[13441:13483]

'TCAGCTGATGCACAATCGTTTTTAAACGGGTTTGCGGTGTAA'

In [108]:
gen_seq[13441:13497]

'TCAGCTGATGCACAATCGTTTTTAAACGGGTTTGCGGTGTAAGTGCAGCCCGTCTT'

In [109]:
gen_seq[13467:13497]

'CGGGTTTGCGGTGTAAGTGCAGCCCGTCTT'

In [110]:
gen_seq[13441:13468]+gen_seq[13467:13497]

'TCAGCTGATGCACAATCGTTTTTAAACCGGGTTTGCGGTGTAAGTGCAGCCCGTCTT'

In [111]:
Bio.Seq._translate_str(gen_seq[13441:13468]+gen_seq[13467:13497], table)

'SADAQSFLNRVCGVSAARL'

In [112]:
gen_seq[13441:13497]

'TCAGCTGATGCACAATCGTTTTTAAACGGGTTTGCGGTGTAAGTGCAGCCCGTCTT'

In [113]:
Bio.Seq._translate_str(gen_seq[13441:13497], table)

/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/Bio/Seq.py:2855: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


'SADAQSFLNGFAV*VQPV'

In [114]:
variants

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
0,SRR11578269,NC_045512.2,2668,C,T,6019,0.050008,1,"3670,2007,206,107",MODIFIER,NONE,.,.,.,S,5,0.093522
1,SRR11578028,NC_045512.2,17239,C,T,6139,0.050008,1,"3566,2191,200,127",MODIFIER,NONE,.,.,.,S,9,0.098220
2,SRR11578268,NC_045512.2,26010,C,T,6579,0.050008,7,"4929,1301,276,60",MODIFIER,NONE,.,.,.,S,14,0.087375
3,SRR11578094,NC_045512.2,12050,A,T,5159,0.050010,13,"2507,2381,155,114",MODIFIER,NONE,.,.,.,S,2,0.172133
4,SRR11578157,NC_045512.2,8739,C,T,3079,0.050016,2,"1963,625,125,35",MODIFIER,NONE,.,.,.,S,10,0.054687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68561,SRR11494648,NC_045512,28657,C,T,177,1.000000,0,"0,0,75,102",LOW,SILENT,SYNONYMOUS_CODING,N,gaC/gaT,S,52,0.737106
68562,SRR11605107,NC_045512.2,28881,G,A,102,1.000000,0,"0,0,44,58",MODIFIER,NONE,.,.,.,S,120,0.896983
68563,SRR11605107,NC_045512.2,28882,G,A,103,1.000000,0,"0,0,44,59",MODIFIER,NONE,.,.,.,S,118,0.881007
68564,SRR11605107,NC_045512.2,28883,G,C,102,1.000000,0,"0,0,43,59",MODIFIER,NONE,.,.,.,S,118,0.917067


In [115]:
sadaqsflnr vcgvsaarlt pcgtgtstdv vyrafdiynd kvagfakflk tnccrfqekd

SyntaxError: invalid syntax (<ipython-input-115-bd4a443d9a0d>, line 1)

In [116]:
test=variants[(variants.POS<13497) & (variants.POS>13441)]

In [ ]:
TCA GCT GAT GCA CAA TCG TTT TTA AAC <>  GGG TTT GCG GTG TAA GTG CAG CCC GTC 
S    A   D   A   Q   S   F   L   N       G   F   A   V   *   V   Q   P   V

In [ ]:
TCA GCT GAT GCA CAA TCG TTT TTA AAC <> CGG GTT TGC GGT GTA AGT GCA GCC 
s    a   d   a   q   s   f   l   n      r   v   c   g   v   s   a   a

In [919]:
Bio.Seq._translate_str('GGG', table)

'G'

In [912]:
gen_seq[13469:13472]

'GGT'

In [913]:
(13469-13441)%3

1

In [914]:
(13469-13467)%3

2

In [898]:
Bio.Seq._translate_str(gen_seq[13441:13468], table)

'SADAQSFLN'

In [900]:
Bio.Seq._translate_str(gen_seq[13467:13497], table)

'RVCGVSAARL'

In [853]:
workingtable['MainFeat'].unique()

array(['N', 'ORF3a', 'ORF1a', 'ORF1b', 'S', 'ORF7a', 'ORF8', 'M'],
      dtype=object)

In [854]:
list(workingtable['SecondFeat'].unique())

['na',
 'nsp2',
 '3C-like proteinase',
 'nsp6',
 'RdRp',
 'nsp3',
 'nsp4',
 'leader protein',
 'helicase',
 "3'-to-5' exonuclease"]

## Add Feature information to variant tables

In [118]:
workingtable.head(5)

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,ThirdFeat,ThirdResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
0,0,2,0,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na
1,0,1,0,0,25689,25687,ORF3a 99,Ggc,Agc,G,...,na,na,na,na,na,na,na,na,na,na
2,0,2,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,...,na,na,na,na,na,na,na,na,na,na
3,0,2,0,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,...,na,na,na,na,na,na,na,na,na,na
4,0,2,0,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,...,na,na,na,na,na,na,na,na,na,na


In [119]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
371,SRR11059946,NC_045512,21641,G,C,3780,0.050265,42,"626,2947,14,182",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gca/Cca,S,1,0.050265
396,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gtt/Ttt,S,1,0.050279
397,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Ggc/Tgc,S,1,0.050279
455,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Tct/Cct,S,1,0.050314
523,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,cTc/cCc,S,1,0.050360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68544,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68545,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68546,SRR11494718,NC_045512,25563,G,T,125,1.000000,0,"0,0,52,73",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741
68547,SRR11494732,NC_045512,25563,G,T,135,1.000000,0,"0,0,63,72",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965741


In [120]:
variants['syn']=0
variants['non_syn']=1
variants['MainResidue']='na'
variants['MainFeat']='na'
variants['Main_ref_codon']='na'
variants['Main_alt_codon']='na'
variants['Main_ref_residue']='na'
variants['Main_alt_residue']='na'
variants['SecondFeat']='na'
variants['SecondResidue']='na'
variants['Second_ref_codon']='na'
variants['Second_alt_codon']='na'
variants['Second_ref_residue']='na'
variants['Second_alt_residue']='na'
variants['ThirdResidue']='na'
variants['ThirdFeat']='na'
variants['Third_ref_codon']='na'
variants['Third_alt_codon']='na'
variants['Third_ref_residue']='na'
variants['Third_alt_residue']='na'
variants['Genomic_coord_end']=0
variants['Genomic_coord_start']=0
for i,row in variants.iterrows():
    #synonymous.loc[i, 'ref_codon']=row['CODON'].split('/')[0]
    #synonymous.loc[i, 'alt_codon']=row['CODON'].split('/')[1]
    #synonymous.loc[i, 'ref_residue']=Bio.Seq._translate_str(row['CODON'].split('/')[0], table)
    #synonymous.loc[i, 'alt_residue']=','.join([Bio.Seq._translate_str(row['CODON'].split('/')[1], table) for i in row['CODON'].split(',')])
    features=list(features_mat[(features_mat.start<row['POS']) & (features_mat.end>row['POS'])]['gene'])
    if len(features)>0 :
        variants.loc[i, 'MainFeat']=features[0]
        startmain=int(features_mat[features_mat['gene']==features[0]].start)
        if features[0]=='RdRp2':
            variants.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)+9
        else:
            variants.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)
        if (row['POS']-startmain)%3==0:
            startposmain=row['POS']-2
        else :
            startposmain=row['POS']+1-((row['POS']-startmain)%3)
        variants.loc[i, 'Genomic_coord_start']=startposmain
        variants.loc[i, 'Genomic_coord_end']=startposmain+2
        maincod=gen_seq[(startposmain-1):(startposmain-1)+3]
        index=(row['POS']-startmain)%3
        mainaltcod=maincod[:(row['POS']-startposmain)] + row['ALT'] +  maincod[(row['POS']-startposmain)+1:]
        variants.loc[i, 'Main_ref_codon']=maincod
        variants.loc[i, 'Main_alt_codon']=mainaltcod
        variants.loc[i, 'Main_ref_residue']=Bio.Seq._translate_str(maincod, table)
        variants.loc[i, 'Main_alt_residue']=Bio.Seq._translate_str(mainaltcod, table) 
        if len(features)>=2 :
            variants.loc[i, 'SecondFeat']=features[1]
            startsecond=int(features_mat[features_mat['gene']==features[1]].start)
            if features[1]=='RdRp2':
                variants.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)+9
            else:
                variants.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)
            if (row['POS']-startsecond)%3==0:
                startpossec=row['POS']-2
            else :
                startpossec=row['POS']+1-((row['POS']-startsecond)%3)
            variants.loc[i, 'Genomic_coord_start']=startpossec
            variants.loc[i, 'Genomic_coord_end']=startpossec+2
            seccod=gen_seq[(startpossec-1):(startpossec-1)+3]
            index=(row['POS']-startsecond)%3
            secaltcod=seccod[:(row['POS']-startpossec)] + row['ALT'] +  seccod[(row['POS']-startpossec)+1:]
            variants.loc[i, 'Second_ref_codon']=seccod
            variants.loc[i, 'Second_alt_codon']=secaltcod
            variants.loc[i, 'Second_ref_residue']=Bio.Seq._translate_str(seccod, table)
            variants.loc[i, 'Second_alt_residue']=Bio.Seq._translate_str(secaltcod, table) 
        if len(features)>2:
            print(features)
            variants.loc[i, 'ThirdFeat']=features[2]
            startthird=int(features_mat[features_mat['gene']==features[2]].start)
            if features[2]=='RdRp2':
                variants.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)+9
            else:
                variants.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)
            if (row['POS']-startthird)%3==0:
                startposthird=row['POS']-2
            else :
                startposthird=row['POS']+1-((row['POS']-startthird)%3)
            variants.loc[i, 'Genomic_coord_start']=startposthird
            variants.loc[i, 'Genomic_coord_end']=startposthird+2
            thirdcod=gen_seq[(startposthird-1):(startposthird-1)+3]
            index=(row['POS']-startthird)%3
            thirdaltcod=thirdcod[:(row['POS']-startposthird)] + row['ALT'] +  thirdcod[(row['POS']-startposthird)+1:]
            variants.loc[i, 'Third_ref_codon']=thirdcod
            variants.loc[i, 'Third_alt_codon']=thirdaltcod
            variants.loc[i, 'Third_ref_residue']=Bio.Seq._translate_str(thirdcod, table)
            variants.loc[i, 'Third_alt_residue']=Bio.Seq._translate_str(thirdaltcod, table) 
            print(thirdcod)



['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
TGC
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
TGC
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
GTT
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotei

In [121]:
variants

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
0,SRR11578269,NC_045512.2,2668,C,T,6019,0.050008,1,"3670,2007,206,107",MODIFIER,...,A,A,na,na,na,na,na,na,2668,2666
1,SRR11578028,NC_045512.2,17239,C,T,6139,0.050008,1,"3566,2191,200,127",MODIFIER,...,P,S,na,na,na,na,na,na,17241,17239
2,SRR11578268,NC_045512.2,26010,C,T,6579,0.050008,7,"4929,1301,276,60",MODIFIER,...,na,na,na,na,na,na,na,na,26010,26008
3,SRR11578094,NC_045512.2,12050,A,T,5159,0.050010,13,"2507,2381,155,114",MODIFIER,...,K,*,na,na,na,na,na,na,12052,12050
4,SRR11578157,NC_045512.2,8739,C,T,3079,0.050016,2,"1963,625,125,35",MODIFIER,...,T,I,na,na,na,na,na,na,8740,8738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68561,SRR11494648,NC_045512,28657,C,T,177,1.000000,0,"0,0,75,102",LOW,...,na,na,na,na,na,na,na,na,28657,28655
68562,SRR11605107,NC_045512.2,28881,G,A,102,1.000000,0,"0,0,44,58",MODIFIER,...,na,na,na,na,na,na,na,na,28882,28880
68563,SRR11605107,NC_045512.2,28882,G,A,103,1.000000,0,"0,0,44,59",MODIFIER,...,na,na,na,na,na,na,na,na,28882,28880
68564,SRR11605107,NC_045512.2,28883,G,C,102,1.000000,0,"0,0,43,59",MODIFIER,...,na,na,na,na,na,na,na,na,28885,28883


In [122]:
variants

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
0,SRR11578269,NC_045512.2,2668,C,T,6019,0.050008,1,"3670,2007,206,107",MODIFIER,...,A,A,na,na,na,na,na,na,2668,2666
1,SRR11578028,NC_045512.2,17239,C,T,6139,0.050008,1,"3566,2191,200,127",MODIFIER,...,P,S,na,na,na,na,na,na,17241,17239
2,SRR11578268,NC_045512.2,26010,C,T,6579,0.050008,7,"4929,1301,276,60",MODIFIER,...,na,na,na,na,na,na,na,na,26010,26008
3,SRR11578094,NC_045512.2,12050,A,T,5159,0.050010,13,"2507,2381,155,114",MODIFIER,...,K,*,na,na,na,na,na,na,12052,12050
4,SRR11578157,NC_045512.2,8739,C,T,3079,0.050016,2,"1963,625,125,35",MODIFIER,...,T,I,na,na,na,na,na,na,8740,8738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68561,SRR11494648,NC_045512,28657,C,T,177,1.000000,0,"0,0,75,102",LOW,...,na,na,na,na,na,na,na,na,28657,28655
68562,SRR11605107,NC_045512.2,28881,G,A,102,1.000000,0,"0,0,44,58",MODIFIER,...,na,na,na,na,na,na,na,na,28882,28880
68563,SRR11605107,NC_045512.2,28882,G,A,103,1.000000,0,"0,0,44,59",MODIFIER,...,na,na,na,na,na,na,na,na,28882,28880
68564,SRR11605107,NC_045512.2,28883,G,C,102,1.000000,0,"0,0,43,59",MODIFIER,...,na,na,na,na,na,na,na,na,28885,28883


In [123]:
synonymous['syn']=0
synonymous['non_syn']=1
synonymous['MainResidue']='na'
synonymous['MainFeat']='na'
synonymous['Main_ref_codon']='na'
synonymous['Main_alt_codon']='na'
synonymous['Main_ref_residue']='na'
synonymous['Main_alt_residue']='na'
synonymous['SecondFeat']='na'
synonymous['SecondResidue']='na'
synonymous['Second_ref_codon']='na'
synonymous['Second_alt_codon']='na'
synonymous['Second_ref_residue']='na'
synonymous['Second_alt_residue']='na'
synonymous['ThirdResidue']='na'
synonymous['ThirdFeat']='na'
synonymous['Third_ref_codon']='na'
synonymous['Third_alt_codon']='na'
synonymous['Third_ref_residue']='na'
synonymous['Third_alt_residue']='na'
synonymous['Genomic_coord_end']=0
synonymous['Genomic_coord_start']=0
for i,row in synonymous.iterrows():
    #synonymous.loc[i, 'ref_codon']=row['CODON'].split('/')[0]
    #synonymous.loc[i, 'alt_codon']=row['CODON'].split('/')[1]
    #synonymous.loc[i, 'ref_residue']=Bio.Seq._translate_str(row['CODON'].split('/')[0], table)
    #synonymous.loc[i, 'alt_residue']=','.join([Bio.Seq._translate_str(row['CODON'].split('/')[1], table) for i in row['CODON'].split(',')])
    features=list(features_mat[(features_mat.start<row['POS']) & (features_mat.end>row['POS'])]['gene'])
    
    synonymous.loc[i, 'MainFeat']=features[0]
    startmain=int(features_mat[features_mat['gene']==features[0]].start)
    if features[0]=='RdRp2':
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)+9
    else:
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)
    if (row['POS']-startmain)%3==0:
        startposmain=row['POS']-2
    else :
        startposmain=row['POS']+1-((row['POS']-startmain)%3)
    synonymous.loc[i, 'Genomic_coord_start']=startposmain
    synonymous.loc[i, 'Genomic_coord_end']=startposmain+2
    maincod=gen_seq[(startposmain-1):(startposmain-1)+3]
    index=(row['POS']-startmain)%3
    mainaltcod=maincod[:(row['POS']-startposmain)] + row['ALT'] +  maincod[(row['POS']-startposmain)+1:]
    synonymous.loc[i, 'Main_ref_codon']=maincod
    synonymous.loc[i, 'Main_alt_codon']=mainaltcod
    synonymous.loc[i, 'Main_ref_residue']=Bio.Seq._translate_str(maincod, table)
    synonymous.loc[i, 'Main_alt_residue']=Bio.Seq._translate_str(mainaltcod, table) 
    if len(features)>=2 :
        synonymous.loc[i, 'SecondFeat']=features[1]
        startsecond=int(features_mat[features_mat['gene']==features[1]].start)
        if features[1]=='RdRp2':
            synonymous.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)+9
        else:
            synonymous.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)
        if (row['POS']-startsecond)%3==0:
            startpossec=row['POS']-2
        else :
            startpossec=row['POS']+1-((row['POS']-startsecond)%3)
        synonymous.loc[i, 'Genomic_coord_start']=startpossec
        synonymous.loc[i, 'Genomic_coord_end']=startpossec+2
        seccod=gen_seq[(startpossec-1):(startpossec-1)+3]
        index=(row['POS']-startsecond)%3
        secaltcod=seccod[:(row['POS']-startpossec)] + row['ALT'] +  seccod[(row['POS']-startpossec)+1:]
        synonymous.loc[i, 'Second_ref_codon']=seccod
        synonymous.loc[i, 'Second_alt_codon']=secaltcod
        synonymous.loc[i, 'Second_ref_residue']=Bio.Seq._translate_str(seccod, table)
        synonymous.loc[i, 'Second_alt_residue']=Bio.Seq._translate_str(secaltcod, table) 
    if len(features)>2:
        print(features)
        synonymous.loc[i, 'ThirdFeat']=features[2]
        startthird=int(features_mat[features_mat['gene']==features[2]].start)
        if features[2]=='RdRp2':
            synonymous.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)+9
        else:
            synonymous.loc[i, 'ThirdResidue']=math.ceil((row['POS']-(startthird))/3)
        if (row['POS']-startthird)%3==0:
            startposthird=row['POS']-2
        else :
            startposthird=row['POS']+1-((row['POS']-startthird)%3)
        synonymous.loc[i, 'Genomic_coord_start']=startposthird
        synonymous.loc[i, 'Genomic_coord_end']=startposthird+2
        thirdcod=gen_seq[(startposthird-1):(startposthird-1)+3]
        index=(row['POS']-startthird)%3
        thirdaltcod=thirdcod[:(row['POS']-startposthird)] + row['ALT'] +  thirdcod[(row['POS']-startposthird)+1:]
        synonymous.loc[i, 'Third_ref_codon']=thirdcod
        synonymous.loc[i, 'Third_alt_codon']=thirdaltcod
        synonymous.loc[i, 'Third_ref_residue']=Bio.Seq._translate_str(thirdcod, table)
        synonymous.loc[i, 'Third_alt_residue']=Bio.Seq._translate_str(thirdaltcod, table) 
        print(thirdcod)
        


<ipython-input-123-4d08e4da36c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['syn']=0
<ipython-input-123-4d08e4da36c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['non_syn']=1
<ipython-input-123-4d08e4da36c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG
['ORF1a polyprotein', 'RdRp2', 'ORF1b polyprotein']
CGG


In [124]:
synonymous.MainFeat.unique()

array(['surface glycoprotein', 'ORF1b polyprotein', 'ORF1a polyprotein',
       'RdRp2', 'nucleocapsid phosphoprotein', 'envelope protein',
       'ORF3a protein', 'ORF7a protein', 'ORF10 protein', 'ORF6 protein',
       'ORF8 protein', 'membrane glycoprotein', 'ORF7b'], dtype=object)

In [125]:
variants.MainFeat.unique()


array(['ORF1a polyprotein', 'ORF1b polyprotein', 'ORF3a protein', 'RdRp2',
       'surface glycoprotein', 'nucleocapsid phosphoprotein',
       'ORF10 protein', 'ORF8 protein', 'envelope protein',
       'ORF7a protein', 'membrane glycoprotein', 'na', 'ORF6 protein',
       'ORF7b'], dtype=object)

In [126]:
variants[variants['MainFeat']=='RdRp2']

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
7,SRR11578268,NC_045512.2,13506,C,T,8097,0.050019,7,"3549,3911,204,255",MODIFIER,...,C,C,na,na,na,na,na,na,13506,13504
37,SRR11578135,NC_045512.2,15025,G,T,3297,0.050045,7,"1944,1093,133,60",MODIFIER,...,A,S,na,na,na,na,na,na,15027,15025
61,SRR11577859,NC_045512.2,15619,C,T,5134,0.050058,1,"3485,1358,196,71",MODIFIER,...,L,F,na,na,na,na,na,na,15621,15619
62,SRR11578028,NC_045512.2,14670,C,T,4135,0.050060,7,"2826,1072,150,70",MODIFIER,...,V,V,na,na,na,na,na,na,14670,14668
98,SRR11578173,NC_045512.2,15128,G,T,3594,0.050083,2,"942,2391,56,130",MODIFIER,...,C,F,na,na,na,na,na,na,15129,15127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68496,SRR11494491,NC_045512,14805,C,T,133,1.000000,0,"0,0,40,93",LOW,...,Y,Y,na,na,na,na,na,na,14805,14803
68497,SRR11494500,NC_045512,14805,C,T,164,1.000000,0,"0,0,63,101",LOW,...,Y,Y,na,na,na,na,na,na,14805,14803
68498,SRR11605101,NC_045512.2,14805,C,T,133,1.000000,0,"0,0,54,79",MODIFIER,...,Y,Y,na,na,na,na,na,na,14805,14803
68499,SRR11605116,NC_045512.2,14805,C,T,106,1.000000,0,"0,0,40,66",MODIFIER,...,Y,Y,na,na,na,na,na,na,14805,14803


In [127]:
synonymous[synonymous['MainFeat']=='RdRp2']

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,Second_ref_residue,Second_alt_residue,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start
904,SRR11476446,NC_045512,14757,C,A,178,0.050562,0,"67,101,4,6",MODERATE,...,H,Q,na,na,na,na,na,na,14757,14755
1839,SRR11494657,NC_045512,15193,G,T,137,0.051095,0,"64,66,3,4",MODERATE,...,A,S,na,na,na,na,na,na,15195,15193
1916,SRR11494646,NC_045512,14197,G,T,176,0.051136,1,"86,81,4,5",MODERATE,...,A,S,na,na,na,na,na,na,14199,14197
2193,SRR11494664,NC_045512,14294,G,T,195,0.051282,13,"68,117,7,3",MODERATE,...,R,L,na,na,na,na,na,na,14295,14293
2195,SRR11494646,NC_045512,15202,G,T,156,0.051282,1,"67,80,3,5",MODERATE,...,V,L,na,na,na,na,na,na,15204,15202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68475,SRR11494738,NC_045512,14408,C,T,184,1.000000,0,"0,0,91,93",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
68476,SRR11494496,NC_045512,14408,C,T,202,1.000000,0,"0,0,97,105",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
68477,SRR11494505,NC_045512,14408,C,T,189,1.000000,0,"0,0,92,97",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407
68478,SRR11479040,NC_045512,14408,C,T,211,1.000000,0,"0,0,105,106",MODERATE,...,P,L,na,na,na,na,na,na,14409,14407


### Select relevant collumns

In [128]:
work2=synonymous[['POS', 'REF', 'ALT', 'AF','type', 'syn',
       'non_syn','MainResidue', 'MainFeat', 'Main_ref_codon',
       'Main_alt_codon', 'Main_ref_residue', 'Main_alt_residue', 'SecondFeat',
       'SecondResidue', 'Second_ref_codon', 'Second_alt_codon',
       'Second_ref_residue', 'Second_alt_residue', 'ThirdResidue', 'ThirdFeat',
       'Third_ref_codon', 'Third_alt_codon', 'Third_ref_residue',
       'Third_alt_residue', 'Genomic_coord_end', 'Genomic_coord_start','NbSample','MeanAF']]

In [129]:
workall=variants[['POS', 'REF', 'ALT', 'AF','type', 'syn',
       'non_syn','MainResidue', 'MainFeat', 'Main_ref_codon',
       'Main_alt_codon', 'Main_ref_residue', 'Main_alt_residue', 'SecondFeat',
       'SecondResidue', 'Second_ref_codon', 'Second_alt_codon',
       'Second_ref_residue', 'Second_alt_residue', 'ThirdResidue', 'ThirdFeat',
       'Third_ref_codon', 'Third_alt_codon', 'Third_ref_residue',
       'Third_alt_residue', 'Genomic_coord_end', 'Genomic_coord_start','NbSample','MeanAF']]

In [130]:
workall

,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,Main_ref_codon,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
0,2668,C,T,0.050008,S,0,1,801,ORF1a polyprotein,GCC,...,na,na,na,na,na,na,2668,2666,5,0.093522
1,17239,C,T,0.050008,S,0,1,1258,ORF1b polyprotein,CCT,...,na,na,na,na,na,na,17241,17239,9,0.098220
2,26010,C,T,0.050008,S,0,1,206,ORF3a protein,TAC,...,na,na,na,na,na,na,26010,26008,14,0.087375
3,12050,A,T,0.050010,S,0,1,3929,ORF1a polyprotein,AAG,...,na,na,na,na,na,na,12052,12050,2,0.172133
4,8739,C,T,0.050016,S,0,1,2825,ORF1a polyprotein,ACT,...,na,na,na,na,na,na,8740,8738,10,0.054687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68561,28657,C,T,1.000000,S,0,1,128,nucleocapsid phosphoprotein,GAC,...,na,na,na,na,na,na,28657,28655,52,0.737106
68562,28881,G,A,1.000000,S,0,1,203,nucleocapsid phosphoprotein,AGG,...,na,na,na,na,na,na,28882,28880,120,0.896983
68563,28882,G,A,1.000000,S,0,1,203,nucleocapsid phosphoprotein,AGG,...,na,na,na,na,na,na,28882,28880,118,0.881007
68564,28883,G,C,1.000000,S,0,1,204,nucleocapsid phosphoprotein,GGA,...,na,na,na,na,na,na,28885,28883,118,0.917067


In [131]:
work2

,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,Main_ref_codon,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
371,21641,G,C,0.050265,S,0,1,27,surface glycoprotein,GCA,...,na,na,na,na,na,na,21643,21641,1,0.050265
396,23903,G,T,0.050279,S,0,1,781,surface glycoprotein,GTT,...,na,na,na,na,na,na,23905,23903,1,0.050279
397,24131,G,T,0.050279,S,0,1,857,surface glycoprotein,GGC,...,na,na,na,na,na,na,24133,24131,1,0.050279
455,19159,T,C,0.050314,S,0,1,1898,ORF1b polyprotein,TCT,...,na,na,na,na,na,na,19161,19159,1,0.050314
523,11751,T,C,0.050360,S,0,1,3829,ORF1a polyprotein,CTC,...,na,na,na,na,na,na,11752,11750,1,0.050360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68544,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965741
68545,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965741
68546,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965741
68547,25563,G,T,1.000000,S,0,1,57,ORF3a protein,CAG,...,na,na,na,na,na,na,25563,25561,90,0.965741


In [132]:
workall=workall.drop_duplicates()

In [133]:
work2=work2.drop_duplicates()

In [134]:
workingtable['POS']=workingtable['Genomic_coord_start']
workingtable['REF']='na'
workingtable['AF']='0'
workingtable['type']='NS'

In [135]:
for i,row in workingtable.iterrows():
    if int(row.non_syn)>0:
        workingtable.loc[i, 'type']='S'

Select only non synonymous mutations

In [139]:
workingtable


,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,POS,AF,type
0,0,2,na,0,14409,14407,"ORF1b 313,RdRp 323",cCt,cTt,P,...,na,na,na,na,na,na,na,14407,0,S
1,0,1,na,0,25689,25687,ORF3a 99,Ggc,Agc,G,...,na,na,na,na,na,na,na,25687,0,S
2,0,2,na,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,...,na,na,na,na,na,na,na,1058,0,S
3,0,2,na,0,13731,13729,"ORF1b 87,RdRp 97",gCt,gTt,A,...,na,na,na,na,na,na,na,13729,0,S
4,0,2,na,0,17748,17746,"ORF1b 1426,helicase 504",cCt,cTt,P,...,na,na,na,na,na,na,na,17746,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,1,na,0,16536,16534,"ORF1b 1022,helicase 100",0,0,S,...,na,na,na,na,na,na,na,16534,0,S
124,0,1,na,0,17127,17125,"ORF1b 1219,helicase 297",0,0,L,...,na,na,na,na,na,na,na,17125,0,S
127,0,1,na,0,17886,17884,"ORF1b 1472,helicase 550",0,0,T,...,na,na,na,na,na,na,na,17884,0,S
129,0,1,na,0,21243,21241,"ORF1b 2591,2'-O-ribose methyltransferase 195",0,0,T,...,na,na,na,na,na,na,na,21241,0,S


In [138]:
workingtable[workingtable.Genomic_coord_start==14407]

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Second_alt_codon,Second_ref_residue,Second_alt_residue,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,POS,AF,type


Concatenate Variant Tables

In [ ]:
total=pd.concat([work2,workingtable])

In [971]:
totalall=pd.concat([workall,workingtable])

In [972]:
totalall.POS=totalall.POS.astype('int64')

In [877]:
total.POS=total.POS.astype('int64')

In [878]:
total=total.sort_values(by=['POS']).reset_index()

In [973]:
totalall=totalall.sort_values(by=['POS']).reset_index()

In [974]:
totalall.head(20)


,index,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
0,3120,27,A,T,0.092379,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.092379
1,2831,34,A,T,0.132399,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.132399
2,2645,40,C,T,0.18797,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.187970
3,2724,72,G,A,0.160397,S,0,1,na,na,...,na,na,na,na,na,na,0,0,2,0.109120
4,3822,72,G,A,0.057842,S,0,1,na,na,...,na,na,na,na,na,na,0,0,2,0.109120
5,3243,78,T,G,0.083333,S,0,1,na,na,...,na,na,na,na,na,na,0,0,2,0.067214
6,4107,78,T,G,0.051095,S,0,1,na,na,...,na,na,na,na,na,na,0,0,2,0.067214
7,3681,94,G,C,0.0625,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.062500
8,2325,140,C,T,0.630137,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.630137
9,3472,157,C,T,0.071429,S,0,1,na,na,...,na,na,na,na,na,na,0,0,1,0.071429


In [975]:
totalall['MainFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in totalall.MainFeat]]
totalall['MainFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in totalall.MainFeat]]
totalall['MainFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in totalall.MainFeat]]
totalall['MainFeat']=[re.sub(r"S glycoprotein","S",j) for j in [re.sub(r"membrane",'M',i) for i in totalall.MainFeat]]

In [880]:
total['MainFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.MainFeat]]
total['MainFeat']=[re.sub(r"S glycoprotein","S",j) for j in [re.sub(r"membrane",'M',i) for i in total.MainFeat]]

In [977]:
totalall['MainFeat'].unique()

array(['na', 'ORF1a', 'RdRp2', 'ORF1b', 'S', 'ORF3a', 'E', 'M', 'ORF6',
       'ORF7a', 'ORF7b', 'ORF8', 'N', 'ORF10'], dtype=object)

In [980]:
totalall['SecondFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in totalall.SecondFeat]]
totalall['SecondFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in totalall.SecondFeat]]
totalall['SecondFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in totalall.SecondFeat]]
totalall['SecondFeat']=[re.sub(r"membrane",'M',i) for i in totalall.SecondFeat]

In [981]:
total['SecondFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.SecondFeat]]
total['SecondFeat']=[re.sub(r"membrane",'M',i) for i in total.SecondFeat]

In [978]:
list(set(totalall.SecondFeat.unique())|set(totalall.MainFeat.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 '3C-like proteinase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp10',
 'RdRp1',
 'N',
 'ORF1b',
 'RdRp2',
 "3'-to-5' exonuclease",
 'nsp7',
 "2'-O-ribose methyltransferase",
 'nsp6',
 'ORF1b polyprotein',
 'ORF8',
 'nsp3',
 'endoRNAse',
 'leader protein',
 'ORF6',
 'ORF1a',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a']

In [883]:
list(set(total.SecondFeat.unique())|set(total.MainFeat.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp10',
 'RdRp1',
 'N',
 'ORF1b',
 'RdRp2',
 'nsp7',
 "3'-to-5' exonuclease",
 'leader',
 'nsp6',
 "2'-O-ribose methyltransferase",
 'ORF8',
 'nsp3',
 'endoRNAse',
 'ORF6',
 'ORF1a',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a',
 '3C-likease']

In [982]:
totalall['ThirdFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in totalall.ThirdFeat]]
totalall['ThirdFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in totalall.ThirdFeat]]
totalall['ThirdFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in totalall.ThirdFeat]]
totalall['ThirdFeat']=[re.sub(r"membrane",'M',i) for i in totalall.ThirdFeat]

In [983]:
total['ThirdFeat']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.ThirdFeat]]
total['ThirdFeat']=[re.sub(r"membrane",'M',i) for i in total.ThirdFeat]

In [984]:
list(set(totalall.SecondFeat.unique())|set(totalall.MainFeat.unique())|set(totalall.ThirdFeat.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp10',
 'RdRp1',
 'N',
 'ORF1b',
 'RdRp2',
 'nsp7',
 "3'-to-5' exonuclease",
 'leader',
 'nsp6',
 "2'-O-ribose methyltransferase",
 'ORF8',
 'nsp3',
 'endoRNAse',
 'ORF6',
 'ORF1a',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a',
 '3C-likease']

In [985]:
totalall.columns

Index(['index', 'POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn',
       'MainResidue', 'MainFeat', 'Main_ref_codon', 'Main_alt_codon',
       'Main_ref_residue', 'Main_alt_residue', 'SecondFeat', 'SecondResidue',
       'Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue',
       'Second_alt_residue', 'ThirdResidue', 'ThirdFeat', 'Third_ref_codon',
       'Third_alt_codon', 'Third_ref_residue', 'Third_alt_residue',
       'Genomic_coord_end', 'Genomic_coord_start', 'NbSample', 'MeanAF'],
      dtype='object')

In [887]:
#total=total[['POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn', 'ref_codon', 'alt_codon', 'ref_residue', 'alt_residue', 'Genomic_coord_start', 'Genomic_coord_end', 'cleanMain', 'MainResidue','cleanSecond', 'SecondResidue','cleanThird','ThirdResidue','NbSample','MeanAF']]


Create Tables for each Features

In [951]:
total[total.Genomic_coord_start==14407]

,index,POS,REF,ALT,AF,type,syn,non_syn,MainResidue,MainFeat,...,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF
1066,1075,14408,C,T,0.973214,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1067,785,14408,C,T,0.978947,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1068,1272,14408,C,T,0.968586,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1069,489,14408,C,T,0.985075,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1070,1234,14408,C,T,0.969276,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,694,14408,C,T,0.980769,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1218,261,14408,C,T,0.990338,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1219,927,14408,C,T,0.976046,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195
1220,770,14408,C,T,0.979275,S,0,1,323,RdRp2,...,na,na,na,na,na,na,14409,14407,192,0.979195


In [986]:
dicfeaturesall={}
for i in list(set(totalall.SecondFeat.unique())|set(totalall.MainFeat.unique())|set(totalall.ThirdFeat.unique())) :
    dicfeaturesall[i]=totalall[(totalall.MainFeat==i) | (totalall.SecondFeat==i)| (totalall.ThirdFeat==i)]
    dicfeaturesall[i]=dicfeaturesall[i].drop(labels=['syn', 'non_syn','index'], axis=1)
    if np.all([[dicfeaturesall[i].ThirdFeat.unique()==['na']]]):
        dicfeaturesall[i]=dicfeaturesall[i].drop(labels=['ThirdResidue', 'ThirdFeat', 'Third_ref_codon','Third_alt_codon', 'Third_ref_residue', 'Third_alt_residue'], axis=1)
        if np.all([[dicfeaturesall[i].SecondFeat.unique()==['na']]]):
            dicfeaturesall[i]=dicfeaturesall[i].drop(labels=['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue'], axis=1)

In [889]:
dicfeatures={}
for i in list(set(total.SecondFeat.unique())|set(total.MainFeat.unique())|set(total.ThirdFeat.unique())) :
    dicfeatures[i]=total[(total.MainFeat==i) | (total.SecondFeat==i)| (total.ThirdFeat==i)]
    dicfeatures[i]=dicfeatures[i].drop(labels=['syn', 'non_syn','index'], axis=1)
    if np.all([[dicfeatures[i].ThirdFeat.unique()==['na']]]):
        dicfeatures[i]=dicfeatures[i].drop(labels=['ThirdResidue', 'ThirdFeat', 'Third_ref_codon','Third_alt_codon', 'Third_ref_residue', 'Third_alt_residue'], axis=1)
        if np.all([[dicfeatures[i].SecondFeat.unique()==['na']]]):
            dicfeatures[i]=dicfeatures[i].drop(labels=['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue'], axis=1)

In [990]:
dicfeaturesall['RdRp2'][['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue']]

,SecondFeat,SecondResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue
1699,RdRp2,10,CGG,CTG,R,L
1700,RdRp2,11,GTT,TTT,V,F
1701,ORF1b,13,TGC,GGC,C,G
1702,ORF1b,14,GGC,TGC,G,C
1704,ORF1b,35,GGT,GTT,G,V
...,...,...,...,...,...,...
2106,ORF1b,907,TGG,TGT,W,C
2107,ORF1b,907,TGG,TGT,W,C
2108,ORF1b,914,GCT,TCT,A,S
2109,ORF1b,919,CAT,TAT,H,Y


In [989]:
dicfeatures['RdRp2'][['SecondFeat', 'SecondResidue','Second_ref_codon', 'Second_alt_codon', 'Second_ref_residue','Second_alt_residue']]

,SecondFeat,SecondResidue,Second_ref_codon,Second_alt_codon,Second_ref_residue,Second_alt_residue
998,RdRp2,10,CGG,CTG,R,L
999,ORF1b,13,TGC,GGC,C,G
1000,ORF1b,14,GGC,TGC,G,C
1002,ORF1b,35,GGT,GTT,G,V
1003,ORF1b,35,GGT,GTT,G,V
...,...,...,...,...,...,...
1298,ORF1b,907,TGG,TGT,W,C
1299,ORF1b,907,TGG,TGT,W,C
1300,ORF1b,914,GCT,TCT,A,S
1301,ORF1b,919,CAT,TAT,H,Y


In [893]:
dicfeatures['RdRp']=pd.concat([dicfeaturesall['RdRp'],dicfeaturesall['RdRp2']])
dicfeatures['RdRp']=pd.concat([dicfeaturesall['RdRp'],dicfeaturesall['RdRp1']])

In [991]:
dicfeaturesall['RdRp']=pd.concat([dicfeaturesall['RdRp'],dicfeaturesall['RdRp2']])
dicfeaturesall['RdRp']=pd.concat([dicfeaturesall['RdRp'],dicfeaturesall['RdRp1']])

In [992]:
dicfeaturesall['RdRp'].SecondResidue

1703     25
1712     63
1723     97
1785    323
1965    449
       ... 
2109    919
2110    919
1696      3
1697      5
1698      6
Name: SecondResidue, Length: 415, dtype: object

In [993]:
for i in dicfeaturesall.keys():
    dicfeaturesall[i].to_csv(i+'_all_var.csv',  sep='\t',index=False, header=True)

In [895]:
for i in dicfeatures.keys():
    dicfeatures[i].to_csv(i+'_var.csv',  sep='\t',index=False, header=True)

In [994]:
dicfeaturesall['RdRp'][dicfeaturesall['RdRp'].Second_ref_codon!=dicfeaturesall['RdRp'].Second_alt_codon]

,POS,REF,ALT,AF,type,MainResidue,MainFeat,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
1699,13469,G,T,0.053571,S,4402,ORF1a,GGG,TGG,G,...,13470,13468,2,0.053571,1,ORF1b,CGG,CTG,R,L
1700,13471,G,T,0.063218,S,4402,ORF1a,GGG,GGT,G,...,13473,13471,1,0.063218,2,ORF1b,GTT,TTT,V,F
1701,13504,T,G,0.696474,S,22,RdRp2,TGC,GGC,C,...,13506,13504,1,0.696474,na,na,na,na,na,na
1702,13507,G,T,0.725441,S,23,RdRp2,GGC,TGC,G,...,13509,13507,1,0.725441,na,na,na,na,na,na
1704,13571,G,T,0.356688,S,44,RdRp2,GGT,GTT,G,...,13572,13570,6,0.180539,na,na,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,16222,C,T,0.969697,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
2110,16222,C,T,0.994012,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
1696,13450,T,C,0.31401,S,4395,ORF1a,GAT,GAC,D,...,13450,13448,1,0.314010,NaN,NaN,NaN,NaN,NaN,NaN
1697,13454,C,T,0.188406,S,4397,ORF1a,CAA,TAA,Q,...,13456,13454,1,0.188406,NaN,NaN,NaN,NaN,NaN,NaN


In [995]:
dicfeaturesall['RdRp'][dicfeaturesall['RdRp'].Main_ref_codon!=dicfeaturesall['RdRp'].Main_alt_codon]

,POS,REF,ALT,AF,type,MainResidue,MainFeat,Main_ref_codon,Main_alt_codon,Main_ref_residue,...,Genomic_coord_end,Genomic_coord_start,NbSample,MeanAF,ThirdResidue,ThirdFeat,Third_ref_codon,Third_alt_codon,Third_ref_residue,Third_alt_residue
1712,13627,na,0,0,S,53,ORF1b,Gac,Tac,D,...,13629,13627,7,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1723,13729,na,0,0,S,87,ORF1b,gCt,gTt,A,...,13731,13729,4,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1785,14407,na,0,0,S,313,ORF1b,cCt,cTt,P,...,14409,14407,8,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1965,14785,na,0,0,S,439,ORF1b,gCt,gTt,A,...,14787,14785,3,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2043,15277,na,0,0,S,603,ORF1b,Cac,Tac,H,...,15279,15277,7,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2109,16222,C,T,0.969697,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
2110,16222,C,T,0.994012,S,928,RdRp2,CAT,TAT,H,...,16224,16222,2,0.981855,na,na,na,na,na,na
1696,13450,T,C,0.31401,S,4395,ORF1a,GAT,GAC,D,...,13450,13448,1,0.314010,NaN,NaN,NaN,NaN,NaN,NaN
1697,13454,C,T,0.188406,S,4397,ORF1a,CAA,TAA,Q,...,13456,13454,1,0.188406,NaN,NaN,NaN,NaN,NaN,NaN
